#  文件流

## 1.open
open函数使用一个可以搜索到的文件名作为强制参数，返回一个打开的文件对象
## 2.打开模式：

| value | description |
| ----- | ----------- | 
| 'r'   | read        |
| 'w'   | write       |
| 'a'   | append      |
| 'b'   | 二进制模式   |
| '+'   | 扩展模式，在该模式下读写同时使用 |

对于如今只模式，我们通常默认的打开的都是文本文件，但是实际上，我们有时候也需要处理二进制文件，则我们需要使用b选项来正确的而处理二进制文件（声音或者图像），使用的原因就是我们的python在利用文本模式打开的时候，会将换行负在不同平台下的标准进行转化，但是在二进制下，我们经常会用到这种的字符串，所以说我们如果使用文本方式来打开我们的二进制文件的话，会将我们的二进制文件进行相应的破坏，所以要使用二进制模式打开二进制文件   

我们还需要知道，在python中我们的打开文件的第三个默认参数是我们的缓冲区的大小，默认是存在缓冲区的,0无缓冲，1默认使用缓冲，大于1的数字设定缓冲区的大小，-1表示使用默认缓冲区大小

**encoding参数的重要性：**  
在讲解之前我先补充关于python3编码的一些注意要点:  
众所周知，python3已经将utf8作为内部的处理编码，并且已经将python2红的编码方式摒弃选择了更加清晰明了的str和bytes类型,也就是说，在python3中的编码方式实际上会经过这样的一个转化   
**一种编码----->str(unicode)----->另一种编码**  
所以我们如果打开文件的方式encoding选择错的话(编码产生的是字节流，一堆乱码，编码赋予这些字节含义，不同的编码对同一字节流的翻译是不同的),那么我们就会一错误的编码方式去读取文件，就会读取错误的数据，所以为了避免这样，我们需要提前知道我们的要打开的文件的编码，然后打开的时候encoding参数选定这样的编码，这样我们读取的时候才会读到正确的数据，下面会给出实例    

**unicode_escape的重要性**:  
有时候我们会发现文件中的数据都是\uxxx的ｕｎｉｃｏｄｅ编码，但是按照正常的我们的文件的打开的话（文件内部写好的都是UTF8或者其他的编码方式）会变成\\\u的形式，所以我们为了避免这种情况正确的吧数据转化出来到正确的编码语言上，执行下面操作  
* w = '\\\u9738\\\u738b\\\u522b\\\u59ec'    # 霸王别姬的中文unicode编码导出之后的情况
* w = w.encode("utf8").decode('unicode_escape')    #　上面的是字符串类型，我们首先将器转换成字节类型并使用正确的编码，然后使用unicode_escape即可完成

In [4]:
f = open('lantian' , 'w' , encoding = 'gb2312')
f.write("我是兰天")
f.close()

# 错误的编码无法正确的打开文件
#f = open("lantian" , 'w' , encoding = 'utf8')
#f.read()

f = open("lantian" , 'r' , encoding = 'gb2312')
print(f.read())

我是兰天


## 3.基本读写方法
write() - 将字符串写入文件
read() - 提供参数，选择读取的内容的字节数,默认全部读取    

随机访问：  
1.seek（offset，whence）:offset偏移量字节数，whence ： 0文件头开始偏移（offset>=0），1当前位置偏移(offset随意),2文件末尾偏移(offset<=0)  
2.tell:返回当前的位置，字节数

按行读写：  
readline - 读取之第一个换行符  
readlines - 读取文件全部，每行按照一个列表元素插入列表并将列表返回
writelines -  将一个字符串列表全体按照行写入文件，但是不添加换行符 

In [14]:
f = open('a','w+')
f.write("Hello World!\n")
f.seek(0,0)
print(f.read(100) , f.tell())
print('-----------------------------')
f.close()
f = open('w','w+')
k = ['something','lantian','somehing']
f.writelines(k)
f.seek(0,0)
print(f.readlines())
f.close()
print('-----------------------------')
f = open('w','w+')
k = ['something\n','lantian\n','somehing\n']
f.writelines(k)
f.seek(0,0)
print(f.readlines())

Hello World!
 13
-----------------------------
['somethinglantiansomehing']
-----------------------------
['something\n', 'lantian\n', 'somehing\n']


## 关闭文件：
按照上面的描述，我们已经知道了，python是将读写的内容韩存在缓冲区中最后进行统一处理的，这也就意味着而一旦我们没有正确关闭文件的话，有可能会导致写入的内容丢失  
1.手动close()  
2.try，finaly语句强制的close  
3.with open('','') as f:  
      pass  
  该语句块会在文件退出语句块的时候自动的执行关闭操作  
 

with as语句块又称上下文管理器，任何一个定义了__enter__和__exit__方法的对象都会在进入语句块的时候自动的执行这两个方法，其中__enter__方法不带参数，进入with语句的时候被调用，返回值绑定在我们的as后的变量的引用上

In [20]:
class test:
    def __init__(self):
        self.hello = "Hello World!"
        self.bye = "Bye!"
    def __enter__(self):
        print("Into the with as coding cell:" , self.hello)
    def __exit__(self,a,b,c):
        print("Out the with as coding cell:" , self.bye)

with test() as k:
    pass

Into the with as coding cell: Hello World!
Out the with as coding cell: Bye!


### 文件迭代：
不仅是open代开的文件对象，fileinput也是，我们完全可以对文件对象进行迭代，知道我们的所有的文件的数据都被读取完之后跳出迭代循环